# Using SQL to Analyze Factbook Data

This guided project is an introductory project into the use of SQL. 

In [3]:
# Installing necessary packages
!conda install -yc conda-forge ipython-sql

Solving environment: done

# All requested packages already installed.



In [9]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Looking at the data
First, to start I'm going to have a look at the kind of data I'm working with. To do this, I'm using SQL queries in the python notebook. I connected directly to the database with the command above and then I printed the first 5 rows of the database with the query below.

I've listed the data dictionary as well. 

* Data Dictionary:
- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's waterarea in square kilometers.
- `population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.



In [10]:
%%sql
SELECT *
    FROM facts
  LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics
Now that I have an idea of what the data looks like, I'm going to calculate some summary statistics for the popluation and population growth columns. 

In [20]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;


 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


### Observations:
- From the output we can see that the minimum population was 0 and the maximum population was 7,256,490,011.
- We could also see the minimum population growth was 0.0 and the maximum population growth was 4.02%. 

One thing I'm skeptical of is that there zero values for some populations. I'm not sure if there are any countries with no population. 

## Explorihng Outliers

Based on the information we saw earlier, I thought it necessary to do some outlier exploration. 

In [24]:
%%sql
SELECT *
    FROM facts
  WHERE population == (SELECT MIN(population)
                          FROM facts
                      );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


### Observations
- Well it looks like the country was Antarctica, which actually would make some sense. According to the [World Factbook](https://www.cia.gov/the-world-factbook/countries/antarctica/#people-and-society) for Antaractica there are no indigenous habitants in Antarctica, but rather some summer-only research stations. 

In [25]:
%%sql
SELECT *
    FROM facts
  WHERE population == (SELECT MAX(population)
                          FROM facts
                      );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


### Observations

Well we can se pretty clearly in the name column that the whole world's population is included in this row. Now that I know this is the case, I'm going to redo some summary statistics to exclude the row for the whole world. 

## Amending the Summary Statistics


In [26]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
    FROM facts
  WHERE name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


## Observations
Now that I've redone the summary statistics I can see that the max population has decreased and the rest of the summary statistics have remained the same. There is a country that's close to 1.4 billion people which I'm assuming to be China. 

## Exploring the Average Populations, Population Growth Metric, Country Area

In [33]:
%%sql
SELECT ROUND(AVG(population), 2) AS avg_pop,
       ROUND(AVG(population_growth), 2) AS avg_pop_growth,
       ROUND(AVG(area), 2) AS avg_area
    FROM facts
  WHERE name != 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_pop_growth,avg_area
32242666.57,1.2,555093.55


## Observations
- We can see that the average population is 32,242,666.57 million people. 
- The average population growth rate is 1.2%
- The average area for a country is 555,093.55 in KM. 

This brings up the next question, which countries are densely population. 

## Identifying Densely Populated Countries
- We're going to query the database for values that are above the average population value
- The second part of the query will look for countries that are below the average area for a country. 

In [37]:
%%sql
SELECT *
    FROM facts
  WHERE population > (SELECT AVG(population)
                          FROM facts
                      WHERE name != 'World'
                     )
    AND area < (SELECT AVG(area)
                   FROM facts
                WHERE name != 'World'
               )
    ORDER BY population DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


## Conclusions
- A lot of these countries are know for being densely population countries. 
- One thing I didn't realize was that the population for Japan was smaller than that of Bangladesh. 
- China doesn't make the list for the reason that they have a massive population, but some other economic powers like t